In [1]:
import os
import warnings
from gensim.models import doc2vec
from gensim.models.doc2vec import TaggedDocument


In [2]:
faqs = [["1", "당해년도 납입액은 수정 가능 한가요?", "네, 당해년도 납입액은 12464 화면 등록전까지 수정 가능합니다."],
        ["2", "대리인통보 대상계좌 기준은 어떻게 되나요?", "모계좌 기준 가장 최근에 개설된 계좌의 관리점에서 조회 됩니다.  의원폐쇄된 자계좌는 조회대상 계좌에서 제외됩니다. 계좌주 계좌가 사절원 계좌가 아닌 경우만 조회됩니다"],
        ["3", "등록가능 단말기수는 어떻게 되나요?", "5대까지 등록 가능입니다."],
        ["4", "모바일계좌개설 가능한 시간은 어떻게 되나요?", "08:00 ~ 20:00(영업일만 가능"],
        ["5", "미국인일때 미국납세자등록번호 작성 방법은 어떻게 되나요?", "계좌주가 미국인일 때 계좌주의 미국납세자등록번호(사회보장번호(Social Security Number), 고용주식별번호(Employer Identification Number), 개인납세자번호(Individual Taxpayer Identification Number))를 기재합니다.."]
]

In [3]:
faqs

[['1', '당해년도 납입액은 수정 가능 한가요?', '네, 당해년도 납입액은 12464 화면 등록전까지 수정 가능합니다.'],
 ['2',
  '대리인통보 대상계좌 기준은 어떻게 되나요?',
  '모계좌 기준 가장 최근에 개설된 계좌의 관리점에서 조회 됩니다.  의원폐쇄된 자계좌는 조회대상 계좌에서 제외됩니다. 계좌주 계좌가 사절원 계좌가 아닌 경우만 조회됩니다'],
 ['3', '등록가능 단말기수는 어떻게 되나요?', '5대까지 등록 가능입니다.'],
 ['4', '모바일계좌개설 가능한 시간은 어떻게 되나요?', '08:00 ~ 20:00(영업일만 가능'],
 ['5',
  '미국인일때 미국납세자등록번호 작성 방법은 어떻게 되나요?',
  '계좌주가 미국인일 때 계좌주의 미국납세자등록번호(사회보장번호(Social Security Number), 고용주식별번호(Employer Identification Number), 개인납세자번호(Individual Taxpayer Identification Number))를 기재합니다..']]

In [4]:
!pip install konlpy

In [5]:
#형태소 분석
import jpype
from konlpy.tag import Kkma

In [6]:
kkma = Kkma()

def tokenize_kkma(doc):
    jpype.attachThreadToJVM()
    token_doc = ['/'.join(word) for word in kkma.pos(doc) ]
    return token_doc


In [ ]:
#자바를 안하면 나오는 에러
#from konlpy.tag import Kkma
#kkma = Kkma()
#def tokenize_kkma(doc):
#    token_doc = ['/'.join(word) for word in kkma.pos(doc) ]
#    return token_doc

/usr/local/lib/python3.6/dist-packages/jpype/_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [7]:
tokenize_kkma(faqs[0][1])

<ipython-input-6-03b68c9b455b>:4: DeprecationWarning: jpype._core.attachThreadToJVM is deprecated, use java.lang.Thread.attach instead
  jpype.attachThreadToJVM()


['당해/NNG',
 '년도/NNM',
 '납입/NNG',
 '액/XSN',
 '은/JX',
 '수정/NNG',
 '가능/NNG',
 '한/MDN',
 '가요/NNG',
 '?/SF']

In [8]:
# 리스트에서 각 문장부분 토큰화
token_faqs = [(tokenize_kkma(row[1]), row[0]) for row in faqs]

# Doc2Vec에서 사용하는 태그문서형으로 변경
tagged_faqs = [TaggedDocument(d, [c]) for d, c in token_faqs]

In [9]:
tagged_faqs

[TaggedDocument(words=['당해/NNG', '년도/NNM', '납입/NNG', '액/XSN', '은/JX', '수정/NNG', '가능/NNG', '한/MDN', '가요/NNG', '?/SF'], tags=['1']),
 TaggedDocument(words=['대리인/NNG', '통보/NNG', '대상/NNG', '계좌/NNG', '기준/NNG', '은/JX', '어떻/VA', '게/ECD', '되/VV', '나요/EFQ', '?/SF'], tags=['2']),
 TaggedDocument(words=['등록/NNG', '가능/NNG', '단말/NNG', '기수/NNG', '는/JX', '어떻/VA', '게/ECD', '되/VV', '나요/EFQ', '?/SF'], tags=['3']),
 TaggedDocument(words=['모바일/NNG', '계좌/NNG', '개설/NNG', '가능/NNG', '하/XSV', 'ㄴ/ETD', '시간/NNG', '은/JX', '어떻/VA', '게/ECD', '되/VV', '나요/EFQ', '?/SF'], tags=['4']),
 TaggedDocument(words=['미국인/NNG', '일/NNG', '때/NNG', '미국/NNP', '납세자/NNG', '등록/NNG', '번호/NNG', '작성/NNG', '방법/NNG', '은/JX', '어떻/VA', '게/ECD', '되/VV', '나요/EFQ', '?/SF'], tags=['5'])]

In [12]:
    # make model
    import multiprocessing
    cores = multiprocessing.cpu_count()
    d2v_faqs = doc2vec.Doc2Vec(vector_size=50, 
                            alpha=0.025,
                            min_alpha=0.025,
                            hs=1,
                            negative=0,
                            dm=0,
                            dbow_words = 1,
                            min_count = 1,
                            workers = cores,
                            seed=0)
    d2v_faqs.build_vocab(tagged_faqs)

    # train document vectors
    for epoch in range(10):
        d2v_faqs.train(tagged_faqs,
                                   total_examples = d2v_faqs.corpus_count,
                                   epochs = d2v_faqs.epochs
                                   )
        d2v_faqs.alpha -= 0.0025 # decrease the learning rate
        d2v_faqs.min_alpha = d2v_faqs.alpha # fix the learning rate, no decay

In [17]:
predict_vector = d2v_faqs.infer_vector(["당해년도 납입액은 수정 가능 한가요?"])

In [18]:
d2v_faqs.docvecs.most_similar([predict_vector], topn=2)

<ipython-input-18-982697d05295>:1: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  d2v_faqs.docvecs.most_similar([predict_vector], topn=2)


[('2', 0.21605531871318817), ('3', 0.10707802325487137)]

In [19]:
test_string = "대리인통보 대상계좌 기준은 어떻게 되나요?"
tokened_test_string = tokenize_kkma(test_string)
tokened_test_string

['대리인/NNG',
 '통보/NNG',
 '대상/NNG',
 '계좌/NNG',
 '기준/NNG',
 '은/JX',
 '어떻/VA',
 '게/ECD',
 '되/VV',
 '나요/EFQ',
 '?/SF']

In [20]:
test_vector = d2v_faqs.infer_vector(tokened_test_string)
d2v_faqs.docvecs.most_similar([test_vector], topn=2)


<ipython-input-20-5a2cc15b9637>:2: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  d2v_faqs.docvecs.most_similar([test_vector], topn=2)


[('1', 0.1448383331298828), ('3', 0.0218462273478508)]